#Generation of upset plots to compare the performance of the different approaches:
- Pseudobulk Reynolds only
- Pseudobulk Alkon only
- Merged no covariable adjusted
- Merged Covariable adjusted

In [0]:
#Load libraries
## Append the library folder
.libPaths(c("/dbfs/home/boriol@almirall.com/my_r_packages/bulkRNASeq_PBMCs_R4.3", .libPaths()))

library(DESeq2)
library(UpSetR)
library(tidyverse)
library(VennDiagram)
library(RColorBrewer)

.libPaths(c("/dbfs/home/jtrincado@almirall.com/my_r_packages/Seurat", .libPaths()))
library(openxlsx)
library(enrichplot)
library(ggplot2)

G_list <- readRDS("/dbfs/mnt/sandbox/RNASeq/PBMCs_IL4/pathways/G_list20240710.rds")
G_list_protein_coding <- G_list %>% filter(transcript_biotype == "protein_coding")
G_list_protein_coding <- G_list_protein_coding %>%unite("gene", ensembl_gene_id:hgnc_symbol, remove = FALSE)

#KC

##Read results
(log2FC >1 and padj <0.05)

In [0]:
alkon_kc<- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_limma_results_kc.xlsx")
rey_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_limma_results_kc.xlsx")
merge_cov_kc_limma <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_results/adjusting_cov_dataset_DEGs/Limma/limma_results_kc.xlsx")

In [0]:
alkon_kc <- alkon_kc %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
rey_kc <- rey_kc %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
merge_cov_kc <- merge_cov_kc_limma %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))

In [0]:
display(alkon_kc)

In [0]:
# DEGs

alkon_kc_filtered <- alkon_kc %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(alkon_kc_filtered)

rey_kc_filtered <- rey_kc %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(rey_kc_filtered)

merge_cov_kc_filtered <- merge_cov_kc %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(merge_cov_kc_filtered)


## KC Upset plot

In [0]:
# Plot upregulated
genes_kc <- list(
  "DEGs_Limma_KC_Alkon_Only" = alkon_kc_filtered$gene, 
  "DEGs_Limma_KC_Reynolds_Only" = rey_kc_filtered$gene, 
  "DEGs_Limma_KC_Merge_Cov_Adjusted" = merge_cov_kc_filtered$gene
)
genes_kc

In [0]:
options(repr.plot.width=1300, repr.plot.height=700)

In [0]:
?upset

In [0]:
upset(
  fromList(genes_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

#KC DIFF/UNDIFF/KC

##Read results
(log2FC >1 and padj <0.05)

In [0]:
merge_cov_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_results/adjusting_cov_dataset_DEGs//Limma/limma_results_kc.xlsx")

merge_cov_diff_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma/alkon_limma_results_DifKC.xlsx")

merge_cov_undiff_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/DEGs/Limma/alkon_limma_results_UndifKC.xlsx")

In [0]:
merge_cov_diff_kc <- merge_cov_diff_kc %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
merge_cov_undiff_kc <- merge_cov_undiff_kc %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
merge_cov_kc <- merge_cov_kc %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))

In [0]:
display(merge_cov_kc)

In [0]:
# DEGs

merge_cov_undiff_kc_filtered <- merge_cov_undiff_kc %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(merge_cov_undiff_kc_filtered)

merge_cov_diff_kc_filtered <- merge_cov_diff_kc %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(merge_cov_diff_kc_filtered)

merge_cov_kc_filtered <- merge_cov_kc %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(merge_cov_kc_filtered)

## KC Upset plot

In [0]:
# Plot upregulated
genes_kc <- list(
  "DEGs_Limma_Undiff_KC" = merge_cov_undiff_kc_filtered$gene, 
  "DEGs_Limma_Diff_KC" = merge_cov_diff_kc_filtered$gene, 
  "DEGs_Limma_KC" = merge_cov_kc_filtered$gene
)
genes_kc

In [0]:
options(repr.plot.width=1100, repr.plot.height=700)

In [0]:
?upset

In [0]:
upset(
  fromList(genes_kc), 
  order.by = "freq",  
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

#KC Pathways

Comparison KC / DIFF/ UNIDFF

##Read results
(log2FC >1 and padj <0.05)

In [0]:
merge_cov_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_results/GSEA/Limma/res_kc_0.05_AR.xlsx",sheet = "Reactome")

merge_cov_diff_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/Limma/res_kc_0.05_AR.xlsx",sheet = "Reactome")

merge_cov_undiff_kc <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_celltypist_results/GSEA/Limma/res_undifkc_0.05_AR.xlsx", sheet = "Reactome")

In [0]:
max(merge_cov_diff_kc$p.adjust)

## KC Upset plot

In [0]:
# Plot upregulated
p_kc <- list(
  "Pathways_KC" = merge_cov_kc$ID, 
  "Pathways_Diff_KC" = merge_cov_diff_kc$ID, 
  "Pathways_Undiff_KC" = merge_cov_undiff_kc$ID
)
p_kc

In [0]:
options(repr.plot.width=1100, repr.plot.height=700)

In [0]:
?upset

In [0]:
upset(
  fromList(p_kc), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))

#Tcells

##Read results
(log2FC >1 and padj <0.05)

In [0]:
alkon_tcell <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Alkon/pseudobulk/alkon_limma_results_tcell.xlsx")

rey_tcell <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/Reynolds/LvsHC/pseudobulk/reynolds_limma_results_tcell.xlsx")

merge_cov_tcell <- read.xlsx("/dbfs/mnt/sandbox/TFM_PAULA/merged_AR_results/adjusting_cov_dataset_DEGs/Limma/limma_results_tcell.xlsx")

In [0]:
alkon_tcell <- alkon_tcell %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
rey_tcell <- rey_tcell %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))
merge_cov_tcell <- merge_cov_tcell %>% inner_join(G_list_protein_coding, by = c("gene" = "hgnc_symbol"))

In [0]:
display(alkon_tcell)

In [0]:
# DEGs

alkon_tcell_filtered <- alkon_tcell %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(alkon_tcell_filtered)

rey_tcell_filtered <- rey_tcell %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(rey_tcell_filtered)

merge_cov_tcell_filtered <- merge_cov_tcell %>% filter(abs(logFC) > 1, adj.P.Val < 0.05) %>% arrange(desc(abs(logFC)))
dim(merge_cov_tcell_filtered)

## KC Upset plot

In [0]:
# Plot upregulated
genes_tcell <- list(
  "DEGs_TCell_Alkon_Only" = alkon_tcell_filtered$gene, 
  "DEGs_TCell_Reynolds_Only" = rey_tcell_filtered$gene, 
  "DEGs_TCell_Merge_Cov_Adjusted" = merge_cov_tcell_filtered$gene
)
genes_tcell

In [0]:
options(repr.plot.width=1300, repr.plot.height=700)

In [0]:
?upset

In [0]:
upset(
  fromList(genes_tcell), 
  order.by = "freq", 
  main.bar.color = "#393D47", 
  sets.bar.color = "darkgrey", 
  text.scale = c(1.5, 1.5, 1.2, 1.2, 1.5, 2))